In [ ]:
import asyncio
import requests
import planet
from planet import Auth, Session, data_filter, reporting
import getpass
from pathlib import Path

In [ ]:
# import custom classes created in this repo
from utils.features_to_json import SearchFeatures
from utils.thumbnail_plot import PreviewPlot

[Makes use of this example](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/Basemaps-API/basemaps_api_introduction.ipynb) but:

- simplifies it as we only have access to one basemap product - "ps_monthly_sen2_normalized_analytic_8b_sr_subscription_{year_month}_mosaic"
- Reads input shapefile to get bounding box coords for one feature for API search - Planet examples do not cover this
- Uses Planets Python SDK and asynchio, so no need to input API key, just login to Planet account


## Setup
Do not need to input API key using this method, just Planet username and password as would use to sign in to [Planet Explorer](https://planet.com/explorer)   
Credentials are then stored while run the rest of this notebook using planet.Session()

In [ ]:
user = input("Planet Username (email): ")
pw = getpass.getpass("Planet Password: ")
auth = Auth.from_login(user, pw)
auth.store()

In [ ]:
# base url for Planet basemaps
BASE_URL = "https://api.planet.com/basemaps/v1/mosaics/"

In [ ]:
# for our trial available Nov 2023 - Oct 2024
BASEMAP_YEAR_MONTH = "2024_10"

## Prepare search features
Read an input search file. In this case instead of request using json for area of interest geometry will pass a string of bounding box coordinates

In [ ]:
INPUT_FEATURES = "inputs/test_shape.shp"

# Initiate instance of custom class. Using bounding box option to simplify
search_features = SearchFeatures(INPUT_FEATURES, bounding_box=True)
# Create the search json - for all features
json_data = search_features.process()

In [ ]:
# select one feature from the json. In this case the input shapefile has a field called "id" and we want id 2.
filtered_feature = search_features.filter_json("id", 1)
# The simplest way to do the basemap search is using bounding box coords in decimal degrees (minx, miny, maxx, maxy)
# that is what will be used in this example
search_box = search_features.json_to_bbox_string(filtered_feature)
# show what this looks like
search_box

## Search for basemap quads for feature of interest
Need to extract the mosaic id and then find the quads

In [ ]:
# We are just searching one month with access to one basemap product, so should be one mosaic id.

basemap_name = f"ps_monthly_sen2_normalized_analytic_8b_sr_subscription_{BASEMAP_YEAR_MONTH}_mosaic"

async with Session() as sess:
    res = await sess.request("GET", f"{BASE_URL}", params={"name__is": basemap_name})
mosaic_json = res.json()

mosaic_id = mosaic_json["mosaics"][0]["id"]

In [ ]:
# Then search for the quads
quads_url = f"{BASE_URL}{mosaic_id}/quads"
async with Session() as sess:
    res = await sess.request("GET", f"{quads_url}", params={"bbox": search_box})
quad_json = res.json()

## Download quads 
The quads are 16 bit tif images and they seem to have 9 bands, but 8 bands of spectral data

In [ ]:
# extract quad id and download link into a dict
downloads_dict = {}
for item in quad_json["items"]:
    downloads_dict[item["id"]] = item["_links"]["download"]

In [ ]:
def download_quad(id_val, link, output_directory, year_month):
    """Download quads to specified directory
    each quad is saved into a <mosaic_name>_<year>_<month> subdirectory"""
    try:
        # Download the file
        response = requests.get(link, stream=True)
        response.raise_for_status()  # Raise an error for bad responses

        mosaic_dir = (
            Path(output_directory)
            / f"ps_monthly_sen2_normalized_analytic_8b_sr_subscription_{year_month}_mosaic"
        )
        mosaic_dir.mkdir(exist_ok=True)

        # Save the file with the new name
        save_path = Path(mosaic_dir / f"{id_val}.tif")
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Image downloaded as: {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download image: {e}")

In [ ]:
# Download images to this repository's outputs directory (could set an absolute path to another dir if required).
for key, val in downloads_dict.items():
    download_quad(key, val, "outputs", BASEMAP_YEAR_MONTH)